In [2]:
import pandas as pd

# Echonest audio features - IMBALANCED LEARNING
## 1. Premise
The hyper-parameters and results for unbalanced classifiers are taken  from the notebooks:
- bin-genre_decision-tree_KNN  (section 2.1)
- multi-genre_decision-tree-KNN   (section 2.2)
- popularity_decision-tree_KNN    (section 2.3)
Below we report their score and AUC curve for comparing them with the balanced data.

We use PCA to project the dimension of the dataset using only the first 2 principal components.

__________________________________
# 2.1 Binary genre - ROCK & JAZZ
__________________________________
## a) Import dataset

In [3]:
binclass_dataset = pd.read_csv('/Users/gaetanoantonicchio/Documents/GitHub/DataMining-2/datasets & labels [cleaned]/echo_genres.csv')
binclass_dataset.set_index('track_id', inplace=True)
binclass_dataset = binclass_dataset[binclass_dataset['genre_top'].isin(['Rock','Jazz'])]
binclass_dataset.head()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,genre_top
track_id,,,,,,,,,
144,0.909011,0.443643,0.641997,0.924092,0.267669,0.089659,128.537,0.788251,Jazz
145,0.235506,0.438672,0.487752,0.716122,0.070359,0.047298,120.790,0.650452,Jazz
146,0.532019,0.417681,0.476422,0.402500,0.172105,0.035361,135.468,0.682397,Jazz
147,0.778410,0.706681,0.866116,0.806703,0.104650,0.065083,120.218,0.917613,Jazz
153,0.988306,0.255661,0.979774,0.973006,0.121342,0.051740,90.241,0.034018,Rock


In [4]:
print('Class distribution before balancing')
pd.concat(
    [binclass_dataset.genre_top.value_counts(),
                round(binclass_dataset.genre_top.value_counts(normalize=True).mul(100),2)],
                axis=1,
                keys=('counts','percentage (%)')
)

Class distribution before balancing


,counts,percentage (%)
Rock,3892,94.17
Jazz,241,5.83


## b) Plot class distribution in 2D

## c) Imbalanced learning:
#### c.1) Undersampling - Random

#### c.2) Condensed Nearest Neighbor (CNN)

#### c.3) Tomek's Link

#### c.4) Oversampling - Random

#### c.5) SMOTE

#### c.6) ADASYN

#### c.7) Class Weight

#### c.8) Ensamble (** not in the program)

_____________________________________________________________________________________________________________________

## d) Decision tree

### d.1) Unbalanced

### d.2) Balanced

###d.3) Compare results

_____________________________________________________________________________________________________________________

## d) KNN

### d.1) Unbalanced

### d.2) Balanced

###d.3) Compare results